In [1]:
import tensorflow as tf
import numpy as np
import PIL
from imgnet32y_RDAE import *
import matplotlib.pyplot as plt
from functools import wraps
from time import time
import tensorflow_decision_forests as tfdf

2023-04-18 19:08:50.921440: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-04-18 19:08:51.686248: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
2023-04-18 19:08:53.283682: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:996] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-04-18 19:08:53.356921: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:996] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA n

In [2]:
def timing(f):
    @wraps(f)
    def wrap(*args, **kw):
        ts = time()
        result = f(*args, **kw)
        te = time()
        print('func:%r args:[%r, %r] took: %2.4f sec' % \
          (f.__name__, args, kw, te-ts))
        return result
    return wrap

In [3]:
IMG_SIZE = 32;
batch_size = 32;
NUM_CLASSES = 10;
size = (IMG_SIZE,IMG_SIZE);

(ds_train, ds_test), ds_info = tfds.load(
    "imagenette", split=['train','validation'], with_info=True, as_supervised=True
)

In [4]:
def resize(image, label):
    resized = tf.image.resize(image, size)
    resized = tf.image.rgb_to_grayscale(resized)
    return resized, label
def onehot(image, label):
    label = tf.one_hot(label, NUM_CLASSES)
    return image, label

In [5]:
ds_train = ds_train.map(resize, num_parallel_calls=8)
ds_train = ds_train.map(onehot, num_parallel_calls=8)
ds_train = ds_train.batch(batch_size=batch_size, drop_remainder=True)

ds_test = ds_test.map(resize, num_parallel_calls=8)
ds_test = ds_test.map(onehot, num_parallel_calls=8)
ds_test = ds_test.batch(batch_size=batch_size, drop_remainder=True)

ds_train = ds_train.prefetch(tf.data.AUTOTUNE)

In [6]:
compressor,decompressor = load_pretrained_RDAE()

2023-04-18 19:08:58.421694: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:424] Loaded cuDNN version 8801
2023-04-18 19:08:59.143293: I tensorflow/tsl/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory
2023-04-18 19:08:59.859239: I tensorflow/compiler/xla/service/service.cc:169] XLA service 0x560fe0609f90 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2023-04-18 19:08:59.859277: I tensorflow/compiler/xla/service/service.cc:177]   StreamExecutor device (0): NVIDIA GeForce RTX 2060, Compute Capability 7.5
2023-04-18 19:08:59.865652: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:269] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2023-04-18 19:08:59.947531: I tensorflow/tsl/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory
2023-04-18 19:09:00.032015: I ./tensorflow/compiler/jit/device_comp

1/1 [==============================] - ETA: 0s - loss: 2261.6477 - distortion_loss: 0.9993 - rate_loss: 263.0054 - distortion_pass_through_loss: 0.9993 - rate_pass_through_loss: 263.0054

1/1 [==============================] - 8s 8s/step - loss: 2261.6477 - distortion_loss: 0.9993 - rate_loss: 263.0054 - distortion_pass_through_loss: 0.9993 - rate_pass_through_loss: 263.0054 - val_loss: 2258.0669 - val_distortion_loss: 0.9992 - val_rate_loss: 259.6149 - val_distortion_pass_through_loss: 0.9992 - val_rate_pass_through_loss: 259.6149


In [7]:
@timing
def encode():
    C = []
    for X,y in ds_train.take(-1):
        strings, entropies = compressor(X)
        C.append(strings)
    C = tf.reshape(tf.stack(C),(9440))
    np.save('imagenette_32y',np.array(C))
encode()

C = []
for X,y in ds_test.take(-1):
    strings, entropies = compressor(X)
    C.append(strings)
C = tf.reshape(tf.stack(C),(3904))
np.save('imagenette_32y_val',np.array(C))

2023-04-18 19:09:03.037814: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_4' with dtype int64 and shape [16]
	 [[{{node Placeholder/_4}}]]
2023-04-18 19:09:03.038280: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_3' with dtype int64 and shape [16]
	 [[{{node Placeholder/_3}}]]


func:'encode' args:[(), {}] took: 5.0109 sec


2023-04-18 19:09:08.048386: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_3' with dtype int64 and shape [4]
	 [[{{node Placeholder/_3}}]]
2023-04-18 19:09:08.049043: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_3' with dtype int64 and shape [4]
	 [[{{node Placeholder/_3}}]]


In [8]:
@timing
def partial_decode():
    C = np.load('imagenette_32y.npy',allow_pickle=True)
    D = decompressor.entropy_model.decompress(C,broadcast_shape=(1,1))
    return D
D = partial_decode()

func:'partial_decode' args:[(), {}] took: 0.0232 sec


In [9]:
@timing
def full_decode():
    C = np.load('imagenette_32y.npy',allow_pickle=True)
    D = decompressor(C)
    return D
D = full_decode()

func:'full_decode' args:[(), {}] took: 1.8046 sec


In [10]:
(ds_train, ds_test), ds_info = tfds.load(
    "imagenette", split=['train','validation'], with_info=True, as_supervised=True
)

X_train = tf.squeeze(partial_decode())

C = np.load('imagenette_32y_val.npy',allow_pickle=True)
X_val = tf.squeeze(decompressor.entropy_model.decompress(C,broadcast_shape=(1,1)))

y_train = []
for _,y in ds_train.take(-1):
    y_train.append(y)
y_train = tf.stack(y_train[0:9440])

y_val = [];
for _,y in ds_test.take(-1):
    y_val.append(y)    
y_val = tf.stack(y_val[0:3904])

func:'partial_decode' args:[(), {}] took: 0.0130 sec


2023-04-18 19:09:12.143472: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_1' with dtype string and shape [16]
	 [[{{node Placeholder/_1}}]]
2023-04-18 19:09:12.143837: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_2' with dtype string and shape [16]
	 [[{{node Placeholder/_2}}]]
2023-04-18 19:09:17.991803: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_3' with dtype int64 and shape [4]
	 [[{{

In [11]:
display(X_train.shape)
display(X_val.shape)
display(y_train.shape)
display(y_val.shape)

TensorShape([9440, 128])

TensorShape([3904, 128])

TensorShape([9440])

TensorShape([3904])

In [12]:
model = tfdf.keras.GradientBoostedTreesModel()
model.fit(x=X_train, y=y_train)
np.sum(np.argmax(model.predict(X_val),axis=1) == y_val)/len(y_val)

Use /tmp/tmpcxqfynf1 as temporary training directory
Reading training dataset...


[WARNING 23-04-18 19:09:20.5150 CDT gradient_boosted_trees.cc:1797] "goss_alpha" set but "sampling_method" not equal to "GOSS".
[WARNING 23-04-18 19:09:20.5150 CDT gradient_boosted_trees.cc:1808] "goss_beta" set but "sampling_method" not equal to "GOSS".
[WARNING 23-04-18 19:09:20.5150 CDT gradient_boosted_trees.cc:1822] "selective_gradient_boosting_ratio" set but "sampling_method" not equal to "SELGB".


Training dataset read in 0:00:05.686583. Found 9440 examples.
Training model...
Model trained in 0:00:24.415089
Compiling model...


[INFO 23-04-18 19:09:50.4993 CDT kernel.cc:1242] Loading model from path /tmp/tmpcxqfynf1/model/ with prefix ddbd9dd3b503455c
[INFO 23-04-18 19:09:50.6060 CDT decision_forest.cc:660] Model loaded with 890 root(s), 41612 node(s), and 128 input feature(s).
[INFO 23-04-18 19:09:50.6060 CDT abstract_model.cc:1311] Engine "GradientBoostedTreesGeneric" built
[INFO 23-04-18 19:09:50.6061 CDT kernel.cc:1074] Use fast generic engine


Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: could not get source code
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: could not get source code
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: could not get source code
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Model compiled.
122/122 [==============================] - 1s 7ms/step


0.3711577868852459

In [20]:
@timing
def train():
    model.fit(x=X_train, y=y_train)
train()

Reading training dataset...
Training dataset read in 0:00:01.428915. Found 9440 examples.
Training model...


[INFO 23-04-18 19:17:42.0926 CDT decision_forest.cc:660] Model loaded with 1160 root(s), 53106 node(s), and 128 input feature(s).


Model trained in 0:00:01.203441
Compiling model...


[INFO 23-04-18 19:17:42.9819 CDT kernel.cc:1242] Loading model from path /tmp/tmpcxqfynf1/model/ with prefix ddbd9dd3b503455c
[INFO 23-04-18 19:17:43.1128 CDT decision_forest.cc:660] Model loaded with 1130 root(s), 51810 node(s), and 128 input feature(s).
[INFO 23-04-18 19:17:43.1129 CDT kernel.cc:1074] Use fast generic engine


Model compiled.
func:'train' args:[(), {}] took: 3.0659 sec
